# Implementing our RAG Application

We've walked through the process of ingesting our data. Now we want to setup our search application.

There are multiple ways of implementing search that we won't be addressing in this workshop.

We're going to implement a similarity search. We can choose to use the OpenSearch®️ SDK or we can use LangChain.

## Similarity Search with Langchain

LangChain allows us to connect our existing configurations across our platform to ensure that we settings are consistent. This is paramount to the success of your similarity search in that if you're search parameters differ from the values you selected to load data from, you can run into some issues.

LangChain gives us the ability to access a [preexisting OpenSearch instance](https://python.langchain.com/docs/integrations/vectorstores/opensearch/#using-a-preexisting-opensearch-instance).

In [1]:
import os
from pprint import pprint

from dotenv import load_dotenv
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import OpenSearchVectorSearch

load_dotenv()

embeddings = HuggingFaceEmbeddings()
vector_search = OpenSearchVectorSearch(
    index_name=os.getenv("INDEX_NAME"),
    embedding_function=embeddings,
    opensearch_url=os.getenv("OPENSEARCH_SERVICE_URI"),
)

query = "How can I be productive"

results = vector_search.similarity_search(
    query,
    vector_field="content_vector",
    text_field="content",
    metadata_field="*",
)

pprint(results[0].page_content)

/Users/jay.miller/preparing-data-for-opensearch-and-rag/.venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/Users/jay.miller/preparing-data-for-opensearch-and-rag/.venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


('. Boy, how do you do? I feel you on this ever since I started just having a '
 'single page to write down what actually gets done and crossing it off, even '
 "if it doesn't get on the list until after I've done something. It's helped "
 "me realize and recognize how much I'm actually getting done")


In [2]:
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOllama(model="llama3")
prompt = ChatPromptTemplate.from_template("""
        Offer supportive advice for the question {query} with supporting quotes from 
        "{docs}".
                                          
        Don't include quotes from other sources.
""")

chain = prompt | llm | StrOutputParser()

print(chain.invoke({"query": query, "docs": "\n".join([result.page_content for result in results])}))
print("-------------------")
print(f'Here are some episodes that might help you with "{query}":')
episodes = set()
for result in results:
    episodes.add(f"{result.metadata["title"]} - {result.metadata["url"]}")
print("\n".join(episodes))

Here's some supportive advice on how to be productive while dealing with others:

Firstly, recognizing your strengths and accomplishments, as you mentioned, is a great step towards boosting your productivity. By keeping track of what you've achieved, you can gain confidence and motivation to tackle more tasks.

To set boundaries with others and maintain your focus during high-productivity periods, consider the following strategies:

1. **Communicate your needs**: Let others know when you're in a focused work mode and prefer minimal interruptions. You can say something like, "I'm on a deadline for this project, so I'd appreciate it if we could touch base later."
2. **Schedule breaks**: Plan regular breaks to recharge and respond to messages or calls from others. This way, you'll be more present and energized during your focused work sessions.
3. **Establish routines**: Develop consistent routines for checking emails, taking phone calls, or meeting with others. This can help you avoid co

This is good but we probably don't want to use Ollama in our production environment. Let look at how easy it is to use a different model.

> NOTE: ⚠️ The next block uses the OpenAI API which cannot be used without an API key which you will need to pay for.

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

load_dotenv()

llm = ChatOpenAI()
prompt = ChatPromptTemplate.from_messages([
    ("system",
     """Offer supportive advice for the question {query} with supporting quotes from 
     ---
     {docs}
     ---
     Wrap quotes in quotation marks. Don't include quotes from other sources.
     If there are no documents to quote, say \"I don't have any information on that.\"""")
    ,
    ("user",
     "{query}"),
])

chain = prompt | llm | StrOutputParser()

print(chain.invoke({"query": query, "docs": "\n".join([result.page_content for result in results])}))
print("-------------------")
print(f'Here are some episodes that might help you with "{query}":')
episodes = set()
for result in results:
    episodes.add(f"{result.metadata["title"]} - {result.metadata["url"]}")
print("\n".join(episodes))

To be productive, it's essential to have a clear plan and track your progress. Start by creating a to-do list and prioritize tasks. As you complete each item, cross it off to see your accomplishments. As mentioned, "having a single page to write down what actually gets done and crossing it off" can help you stay focused and motivated.

Setting boundaries with others is also crucial. Learn to say no to non-urgent interruptions during your high-focus times. As one person shared, "I need to set boundaries for others as I'm constantly interrupted in the depths of my work or high focus times with things that are not urgent." Remember, it's okay to prioritize your own productivity.

Dealing with external factors, especially other people, can be challenging for productivity. Find a balance between helping others and protecting your focus. As mentioned, "how do you handle dealing with other people when it comes to your personal productivity?" Don't let people-pleasing tendencies hinder your ow